* figure 5 有小bug，替换的图放在压缩包里了；


* figure 4 改了下字体，替换的图放在压缩包里了；


* 我觉得section8应该可以放在section6后面，备忘录应该放在section7后面，当然排序这种问题可以讨论；


* Figure 1比后面几个图小一点，可以调整一下；


* 王同学有几个图都没加图号，可以加一下；


* 王同学的那个表可以放在附录里，但是在文章里要说明这个图放在附录里了；


* 像Figure 8这种需要添加Caption的情况，最好告诉添加caption的字数range，这样改一遍就差不多了；


* table 3里的数字和文字能不能都居中，或者保持一致；


* table 2放左边，figure 8放右边视觉效果会更好一些；


* table 1的修改如下：


|Symbol/Abbreviation|Description|
|:---:|:---:|
|$A_1$|Natural gas consumed by the electric|
|$A_2$|Electricity produced from solar energ|
|$A_3$|Electricity produced from wind energy|
|$A_4$|Electricity produced from geothermal energy|
|$A_5$|Hydroelectricity produced by the electric|
|$w$|The importance of the row vector to the column|
|$CR_{j}$|Metrics for Consistency Checks|


* 代码的修改如下：

In [ ]:
import numpy as np
import pandas as pd
import warnings


class AHP:
    def __init__(self, criteria, b):
        self.RI = (0, 0, 0.58, 0.9, 1.12, 1.24, 1.32, 1.41, 1.45, 1.49)
        self.criteria = criteria
        self.b = b
        self.num_criteria = criteria.shape[0]
        self.num_project = b[0].shape[0]

    def cal_weights(self, input_matrix):
        input_matrix = np.array(input_matrix)
        n, n1 = input_matrix.shape
        assert n == n1, 'not a square matrix'
        for i in range(n):
            for j in range(n):
                if np.abs(input_matrix[i, j] * input_matrix[j, i] - 1) > 1e-7:
                    raise ValueError('not an anti-mutually symmetric matrix')

        eigenvalues, eigenvectors = np.linalg.eig(input_matrix)

        max_idx = np.argmax(eigenvalues)
        max_eigen = eigenvalues[max_idx].real
        eigen = eigenvectors[:, max_idx].real
        eigen = eigen / eigen.sum()

        if n > 9:
            CR = None
            warnings.warn('Unable to judge consistency')
        else:
            CI = (max_eigen - n) / (n - 1)
            CR = CI / self.RI[n-1]
        return max_eigen, CR, eigen

    def run(self):
        max_eigen, CR, criteria_eigen = self.cal_weights(self.criteria)
        print('Criterion layer: maximum eigenvalue {:<5f}, CR={:<5f}, 
              test {} passed'\
              .format(max_eigen, CR, '' if CR < 0.1 else 'no'))
        print('Criterion layer weight={}\n'.format(criteria_eigen))

        max_eigen_list, CR_list, eigen_list = [], [], []
        for i in self.b:
            max_eigen, CR, eigen = self.cal_weights(i)
            max_eigen_list.append(max_eigen)
            CR_list.append(CR)
            eigen_list.append(eigen)

        pd_print = pd.DataFrame(eigen_list,
                                index=['criterion' + \\
                                       str(i) for i in \\
                                       range(self.num_criteria)],
                                columns=['plan' + \\
                                         str(i) for i in \\
                                         range(self.num_project)],
                                )
        pd_print.loc[:, 'largest eigenvalue'] = max_eigen_list
        pd_print.loc[:, 'CR'] = CR_list
        pd_print.loc[:, 'Consistency check'] = pd_print.loc[:, 'CR'] < 0.1
        print('Scheme layer')
        print(pd_print)

        # Criterion layer
        obj = np.dot(criteria_eigen.reshape(1, -1), np.array(eigen_list))
        print('\nCriterion layer', obj)
        print('The best scheme is the plan{}'.format(np.argmax(obj)))
        return obj


if __name__ == '__main__':
    # Criterion Importance Matrix
    criteria = np.array([[1, 5, 7, 9, 5],
                         [1 / 5, 1, 3, 5, 1],
                         [1 / 7, 1 / 3, 1, 5, 1 / 5],
                         [1 / 9, 1 / 5, 1 / 5, 1, 1 / 7],
                         [1 / 5, 1, 5, 7, 1]])

    # For each criterion, the programs are ranked according to their merits
    b1 = np.array([[1, 1.71, 3.48, 1.35], 
                   [1 /1.71, 1, 2.03, 0.786],
                   [1 / 3.48, 1 / 2.03, 1, 0.387], 
                   [1 / 1.35, 1 / 0.786, 1 / 0.387, 1]])
    b2 = np.array([[1, 2.84, 9, 9], 
                   [1 / 2.84, 1, 9, 9], 
                   [1 / 9, 1 / 9, 1, 1], 
                   [1 / 9, 1 / 9, 1, 1]])
    b3 = np.array([[1, 9, 1, 0.851], 
                   [1 / 9, 1, 1 / 9, 1 / 9], 
                   [1, 9, 1, 0.851], 
                   [1 / 0.851, 9, 1 / 0.851, 1]])
    b4 = np.array([[1, 9, 9, 9], 
                   [1 / 9, 1, 1, 1], 
                   [1 / 9, 1, 1, 1], 
                   [1 / 9, 1, 1, 1]])
    b5 = np.array([[1, 2.20, 9, 9], 
                   [1 / 2.20, 1, 9, 9], 
                   [1 / 9, 1 / 9, 1, 1.91],
                   [1 / 9, 1 / 9, 1 / 1.91, 1]])

    b = [b1, b2, b3, b4, b5]
    a = AHP(criteria, b).run()